In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [4]:
df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')

In [ ]:
df.head()

In [5]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['SalePrice'])
y = df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape,y_train.shape

In [ ]:
X_train.isna().mean()

In [6]:
import numpy as np
import pandas as pd
from typing import Dict, Union, List, Optional
from sklearn.base import BaseEstimator, TransformerMixin

class DataCleaner(BaseEstimator, TransformerMixin):
    """
    Final corrected version of DataCleaner with consistent variable naming
    """
    
    def __init__(self, 
                 fill_strategies: Optional[Dict[str, Union[str, float, int]]] = None,
                 default_strategy: str = 'mode',
                 default_fill_value: Optional[Union[float, int, str]] = None,
                 drop_threshold: float = 0.8,
                 verbose: bool = True):
        """
        Args:
            fill_strategies: Dictionary of {column: strategy} pairs
            default_strategy: Strategy for unspecified columns
            default_fill_value: Default value when strategy='value'
            drop_threshold: Drop columns with NaN ratio >= this value
            verbose: Whether to print processing details
        """
        self.fill_strategies = fill_strategies or {}
        self.default_strategy = default_strategy
        self.default_fill_value = default_fill_value
        self.drop_threshold = drop_threshold
        self.verbose = verbose
        self._fill_values = {}
        self._dropped_columns = []
        
    def fit(self, X: pd.DataFrame, y=None):
        """Analyze data and compute fill values"""
        
        # Identify columns to drop
        total_rows = len(X)
        self._dropped_columns = [
            col for col in X.columns 
            if X[col].isna().sum() / total_rows >= self.drop_threshold
        ]
        
        
        # Compute fill values for remaining columns
        remaining_cols = [col for col in X.columns if col not in self._dropped_columns]
        df_remaining = X[remaining_cols]
        
        self._fill_values = {}

        
        for col in df_remaining.columns:
            strategy = self.fill_strategies.get(col, self.default_strategy)
            
            if strategy == 'mode':
                fill_value = df_remaining[col].mode()[0] if not df_remaining[col].mode().empty else self.default_fill_value
            elif strategy == 'median':
                fill_value = df_remaining[col].median() if pd.api.types.is_numeric_dtype(df_remaining[col]) else self.default_fill_value
            elif strategy == 'mean':
                fill_value = df_remaining[col].mean() if pd.api.types.is_numeric_dtype(df_remaining[col]) else self.default_fill_value
            elif strategy == 'value':
                fill_value = self.fill_strategies.get(col, self.default_fill_value)
            elif strategy == 'none':
                fill_value = 0
            else:
                raise ValueError(f"Unknown strategy: {strategy}")
            
            self._fill_values[col] = fill_value
            

        return self
    
    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        """Apply the cleaning transformations"""
        # Drop high-NA columns
        df_filled = X.drop(columns=self._dropped_columns, errors='ignore')
        
        # Fill remaining NAs
        for col, fill_value in self._fill_values.items():
            if col in df_filled.columns and fill_value is not None:
                df_filled[col] = df_filled[col].fillna(fill_value)
        
        if self.verbose:
            remaining_na = df_filled.isna().sum().sum()
            print(f"\nPost-cleaning report:")
            print(f"- Dropped {len(self._dropped_columns)} columns")
            print(f"- Remaining NA values: {remaining_na}")
            if remaining_na > 0:
                print("Columns with remaining NAs:")
                print(df_filled.isna().sum()[df_filled.isna().sum() > 0])
        
        return df_filled

In [7]:
cleaner = DataCleaner(
    fill_strategies={
        'GarageYrBlt': 'value'
    },
    default_strategy='mode',
    default_fill_value=0,
    drop_threshold=0.8,
    verbose=True
)

# Clean the data
train_clean = cleaner.fit_transform(X_train)
test_clean = cleaner.transform(X_test)


Post-cleaning report:
- Dropped 4 columns
- Remaining NA values: 0

Post-cleaning report:
- Dropped 4 columns
- Remaining NA values: 0


In [ ]:
train_clean.isna().sum()

In [8]:
train_clean

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
254,255,20,RL,70.0,8400,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,0,6,2010,WD,Normal
1066,1067,60,RL,59.0,7837,Pave,IR1,Lvl,AllPub,Inside,...,40,0,0,0,0,0,5,2009,WD,Normal
638,639,30,RL,67.0,8777,Pave,Reg,Lvl,AllPub,Inside,...,0,164,0,0,0,0,5,2008,WD,Normal
799,800,50,RL,60.0,7200,Pave,Reg,Lvl,AllPub,Corner,...,0,264,0,0,0,0,6,2007,WD,Normal
380,381,50,RL,50.0,5000,Pave,Reg,Lvl,AllPub,Inside,...,0,242,0,0,0,0,5,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,1096,20,RL,78.0,9317,Pave,IR1,Lvl,AllPub,Inside,...,22,0,0,0,0,0,3,2007,WD,Normal
1130,1131,50,RL,65.0,7804,Pave,Reg,Lvl,AllPub,Inside,...,44,0,0,0,0,0,12,2009,WD,Normal
1294,1295,20,RL,60.0,8172,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,0,4,2006,WD,Normal
860,861,50,RL,55.0,7642,Pave,Reg,Lvl,AllPub,Corner,...,240,0,0,0,0,0,6,2007,WD,Normal


In [9]:
train_clean.isna().sum()

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
Length: 76, dtype: int64

In [10]:
cat_cols = [col for col in train_clean.columns if train_clean[col].dtype == 'object']
num_cols = [col for col in train_clean.columns if train_clean[col].dtype != 'object']


In [ ]:
cat_cols

In [11]:
s = train_clean[cat_cols].nunique()

In [12]:
s

MSZoning          5
Street            2
LotShape          4
LandContour       4
Utilities         2
LotConfig         5
LandSlope         3
Neighborhood     25
Condition1        9
Condition2        8
BldgType          5
HouseStyle        8
RoofStyle         6
RoofMatl          7
Exterior1st      15
Exterior2nd      16
MasVnrType        3
ExterQual         4
ExterCond         5
Foundation        6
BsmtQual          4
BsmtCond          4
BsmtExposure      4
BsmtFinType1      6
BsmtFinType2      6
Heating           6
HeatingQC         5
CentralAir        2
Electrical        4
KitchenQual       4
Functional        7
FireplaceQu       5
GarageType        6
GarageYrBlt      95
GarageFinish      3
GarageQual        5
GarageCond        5
PavedDrive        3
SaleType          9
SaleCondition     6
dtype: int64

In [13]:
threshold = 3

woe_columns = list(s[s > 3].index)
one_hot_columns = list(s[s <= 3].index)

In [14]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

class CustomPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, woe_columns, one_hot_columns):
        self.woe_columns = woe_columns  # Columns for WOE encoding
        self.one_hot_columns = one_hot_columns  # Columns for OneHotEncoding

    def fit(self, X, y):
        # Fill strategy for WOE columns
        self.woe_columns_fill_na = X[self.woe_columns].mode().T[0].to_dict()
        
        df_woe = X.copy()
        df_woe['SalePrice'] = y

        woe_mappings = {}
        iv_values = {}

        for col in self.woe_columns:
          

            groups = df_woe.groupby([col])['SalePrice'].agg(['count', 'sum'])
            groups.columns = ['n_obs', 'n_pos']
            groups['n_neg'] = groups['n_obs'] - groups['n_pos']

            groups['prop_pos'] = groups['n_pos'] / groups['n_pos'].sum()
            groups['prop_neg'] = groups['n_neg'] / groups['n_neg'].sum()

            groups['woe'] = np.log(groups['prop_pos'] / groups['prop_neg'])
            groups['iv'] = (groups['prop_pos'] - groups['prop_neg']) * groups['woe']

            groups.replace([np.inf, -np.inf], 0, inplace=True)
            groups.fillna(0, inplace=True)

            woe_dict = groups['woe'].to_dict()
            iv = groups['iv'].sum()

            woe_mappings[col] = woe_dict
            iv_values[col] = iv

        self.woe_mappings = woe_mappings
        self.iv_values = iv_values

        # Fit OneHotEncoder
        self.ohe = OneHotEncoder(handle_unknown='ignore', sparse=False, drop='first', dtype=int)
        self.ohe.fit(X[self.one_hot_columns])

        return self

    def transform(self, X):
        X_transformed = X.copy()

        # WOE Transformation
        print("***")
   
        for col in self.woe_columns:
            X_transformed[f'{col}_woe'] = X_transformed[col].map(self.woe_mappings[col])
            X_transformed.drop(columns=col, inplace=True)

        # One-Hot Encoding
  
        onehot_array = self.ohe.transform(X[self.one_hot_columns])
        onehot_df = pd.DataFrame(onehot_array,
                                 columns=self.ohe.get_feature_names_out(self.one_hot_columns),
                                 index=X.index)

        X_transformed.drop(columns=self.one_hot_columns, inplace=True)
        X_transformed = pd.concat([X_transformed, onehot_df], axis=1)

        # Check for NaNs (mostly WOE-related)
        print("Check Nans")
        n = X_transformed.isna().mean()
        na_cols = list(n[n > 0].index)
        print(na_cols)

        for col in na_cols:
            name, pr = col.split("_", 1)  # Avoid crashing if underscore appears in original column
            if pr != "woe":
                print("Error Related to Nans")

            dic = self.woe_columns_fill_na
            mappings = self.woe_mappings
            X_transformed[col] = X_transformed[col].fillna(mappings[name][dic[name]])

            print(col, name, pr, dic[name], mappings[name][dic[name]])

        return X_transformed


In [15]:
preprocessor = CustomPreprocessor(woe_columns=woe_columns, one_hot_columns=one_hot_columns)

In [16]:
X_train_t = preprocessor.fit_transform(train_clean, y_train)

***
Check Nans
[]


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [17]:
X_train_t.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,Utilities_NoSeWa,LandSlope_Mod,LandSlope_Sev,MasVnrType_BrkFace,MasVnrType_Stone,CentralAir_Y,GarageFinish_RFn,GarageFinish_Unf,PavedDrive_P,PavedDrive_Y
254,255,20,70.0,8400,5,6,1957,1957,0.0,922,...,0,0,0,1,0,1,1,0,0,1
1066,1067,60,59.0,7837,6,7,1993,1994,0.0,0,...,0,0,0,1,0,1,1,0,0,1
638,639,30,67.0,8777,5,7,1910,1950,0.0,0,...,0,0,0,1,0,1,0,1,1,0
799,800,50,60.0,7200,5,7,1937,1950,252.0,569,...,0,0,0,1,0,1,0,1,0,1
380,381,50,50.0,5000,5,6,1924,1950,0.0,218,...,0,0,0,1,0,1,0,1,0,1


In [18]:
X_test_t = preprocessor.transform(test_clean)

***
Check Nans
['RoofMatl_woe', 'Electrical_woe', 'GarageYrBlt_woe']
RoofMatl_woe RoofMatl woe CompShg 1.978151445288834e-08
Electrical_woe Electrical woe SBrkr -1.6186302498252008e-07
GarageYrBlt_woe GarageYrBlt woe value 4.2947155861470925e-06


In [19]:
X_corr = X_train_t.copy()
X_corr['SalePrice'] = y_train

In [20]:
%%time

corr_matrix = X_corr.corr().abs()

CPU times: user 25.4 ms, sys: 1.2 ms, total: 26.6 ms
Wall time: 25.5 ms


In [21]:
corr_matrix

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,LandSlope_Mod,LandSlope_Sev,MasVnrType_BrkFace,MasVnrType_Stone,CentralAir_Y,GarageFinish_RFn,GarageFinish_Unf,PavedDrive_P,PavedDrive_Y,SalePrice
Id,1.000000,0.002216,0.006751,0.033411,0.017880,0.029571,0.022142,0.032560,0.024427,0.011790,...,0.046952,0.022502,0.014623,0.002622,0.006629,0.005114,0.011663,0.026566,0.001273,0.019643
MSSubClass,0.002216,1.000000,0.334194,0.116501,0.029719,0.052768,0.001928,0.036081,0.013443,0.080944,...,0.005846,0.000379,0.025303,0.005681,0.117300,0.040240,0.043762,0.025914,0.068554,0.088081
LotFrontage,0.006751,0.334194,1.000000,0.271799,0.235679,0.048423,0.108272,0.091124,0.193207,0.206957,...,0.068780,0.006096,0.090051,0.090710,0.056685,0.122628,0.207789,0.001699,0.074778,0.308009
LotArea,0.033411,0.116501,0.271799,1.000000,0.102088,0.001625,0.013541,0.017216,0.126098,0.224270,...,0.081352,0.636206,0.078905,0.035987,0.059751,0.012915,0.104743,0.020291,0.007312,0.266204
OverallQual,0.017880,0.029719,0.235679,0.102088,1.000000,0.087599,0.558124,0.538251,0.413083,0.204864,...,0.061911,0.043649,0.295103,0.331644,0.270798,0.209448,0.526241,0.069409,0.218929,0.785555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GarageFinish_RFn,0.005114,0.040240,0.122628,0.012915,0.209448,0.079868,0.318220,0.201225,0.136764,0.038658,...,0.016637,0.034778,0.083945,0.100042,0.169525,1.000000,0.597076,0.055472,0.166312,0.171860
GarageFinish_Unf,0.011663,0.043762,0.207789,0.104743,0.526241,0.185994,0.624154,0.466426,0.285028,0.209390,...,0.035668,0.003764,0.201000,0.229259,0.289541,0.597076,1.000000,0.099098,0.268409,0.513479
PavedDrive_P,0.026566,0.025914,0.001699,0.020291,0.069409,0.081562,0.179453,0.130853,0.076467,0.069314,...,0.026297,0.013032,0.030255,0.026130,0.074219,0.055472,0.099098,1.000000,0.488681,0.082851
PavedDrive_Y,0.001273,0.068554,0.074778,0.007312,0.218929,0.093269,0.425783,0.177603,0.152918,0.186983,...,0.010894,0.008648,0.081513,0.084862,0.336976,0.166312,0.268409,0.488681,1.000000,0.228901


In [22]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd

class SafeCorrelationFilter(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.8):
        self.threshold = threshold
        self.features_to_drop_ = []
    
    def fit(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        
        # Only consider numeric columns for correlation
        numeric_cols = X.select_dtypes(include=np.number).columns
        if len(numeric_cols) < 2:
            return self
            
        corr_matrix = X[numeric_cols].corr().abs()
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        np.fill_diagonal(mask, False)
        
        high_corr = corr_matrix.mask(~mask).stack()
        high_corr_pairs = high_corr[high_corr > self.threshold].reset_index()
        
        self.features_to_drop_ = []
        for _, row in high_corr_pairs.iterrows():
            feat1, feat2 = row['level_0'], row['level_1']
            self.features_to_drop_.append(feat2)  # Simple strategy: drop the second feature
            
        return self
    
    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        return X.drop(columns=list(set(self.features_to_drop_)), errors='ignore')


In [23]:
features_to_drop=['GarageArea', 'SaleCondition_woe', '1stFlrSF', 'Exterior1st_woe', 'MasVnrType_BrkFace', 'TotRmsAbvGrd']

In [24]:
X_filtered = X_train_t.drop(columns=features_to_drop)

In [25]:
print(f"Original shape: {X_train_t.shape}, New shape: {X_filtered.shape}")

Original shape: (1168, 80), New shape: (1168, 74)


In [26]:
X_filtered.shape

(1168, 74)

In [36]:
!pip install dagshub mlflow

In [37]:
import dagshub

dagshub.init(repo_owner='goguaD', repo_name='HW01', mlflow=True)


Initialized MLflow to track repo "goguaD/HW01"

Repository goguaD/HW01 initialized!

In [38]:
experiment_name = "LogisticRegression_Feature_Selection"
run_name = "logistic_regression_rfe_with_scaling"

In [39]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

# Get experiment by name
experiment = client.get_experiment_by_name("LogisticRegression_Feature_Selection")

# Restore it using its ID
if experiment.lifecycle_stage == "deleted":
    client.restore_experiment(experiment.experiment_id)

In [41]:
mlflow.set_experiment("LogisticRegression_Feature_Selection")
with mlflow.start_run(run_name=run_name):
  mlflow.log_param('test', 'test')
  mlflow.log_metric('test', 1)

🏃 View run logistic_regression_rfe_with_scaling at: https://dagshub.com/goguaD/HW01.mlflow/#/experiments/0/runs/0281ecbefca447029d3137c0b085d22d
🧪 View experiment at: https://dagshub.com/goguaD/HW01.mlflow/#/experiments/0


In [42]:
from sklearn.base import BaseEstimator, TransformerMixin

class FinalCustomPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, woe_columns, one_hot_columns, final_columns):
        self.woe_columns = woe_columns # Columns Which Should Be Preprocessed Using WOE
        self.one_hot_columns = one_hot_columns # Columns Which Should Be Preprocessed Using One Hot Encoder

        self.final_columns = final_columns # Columns Selected By RFE

    def fit(self, X, y):
        # Generate Fill Na Values Just in Case
        self.woe_columns_fill_na = X[woe_columns].mode().T[0].to_dict()
        
        df_woe = X.copy()
        SalePrice_col = 'SalePrice'
        df_woe[SalePrice_col] = y

        woe_mappings = {}
        iv_values = {}
        
        for col in self.woe_columns:
            print(f"Processing {col}...")
            
            groups = df_woe.groupby([col])[SalePrice_col].agg(['count', 'sum'])
            groups.columns = ['n_obs', 'n_pos']
            groups['n_neg'] = groups['n_obs'] - groups['n_pos']
            
            groups['prop_pos'] = groups['n_pos'] / groups['n_pos'].sum()
            groups['prop_neg'] = groups['n_neg'] / groups['n_neg'].sum()
            
            groups['woe'] = np.log(groups['prop_pos'] / groups['prop_neg'])
            groups['iv'] = (groups['prop_pos'] - groups['prop_neg']) * groups['woe']
            
            groups.replace([np.inf, -np.inf], 0, inplace=True)
            groups.fillna(0, inplace=True)
            
            woe_dict = groups['woe'].to_dict()
            iv = groups['iv'].sum()
            
            woe_mappings[col] = woe_dict
            iv_values[col] = iv

        self.woe_mappings = woe_mappings
        self.iv_values = iv_values

        return self


    def transform(self, X):
        X_transformed = X.copy()

        # Preprocess WOE Columns
        print("***")
        print("Preprocessing WOE Columns")
        for col in self.woe_columns:
            X_transformed[f'{col}_woe'] = X_transformed[col].map(self.woe_mappings[col])
            X_transformed.drop(columns=col, inplace=True)

        print("Preprocessing One Hot Columns")
        X_transformed = pd.get_dummies(X_transformed, columns=self.one_hot_columns, drop_first=True, dummy_na=True, dtype=int)

        print("Check Nans")
        n = X_transformed.isna().mean()

        na_cols = list(n[n > 0].index)

        print(na_cols)

        for col in na_cols:
            name, pr = col.split("_")
            if pr != "woe":
                print("Error Related to Nans")

            dic = self.woe_columns_fill_na
            mappings = self.woe_mappings
            X_transformed[col] = X_transformed[col].fillna(mappings[name][dic[name]])

            print(col, name, pr, dic[name], mappings[name][dic[name]])
            
        return X_transformed[self.final_columns]

In [ ]:
woe_columns, one_hot_columns

In [ ]:
X_train.head()

In [43]:
class TypeChecker(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
            
        # Convert any remaining string columns to categorical codes
        for col in X.select_dtypes(include='object').columns:
            X[col] = pd.factorize(X[col])[0]
            
        # Verify all numeric
        assert all(pd.api.types.is_numeric_dtype(X[col]) for col in X.columns)
        return X

In [45]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
import mlflow
import mlflow.sklearn

# --- Feature Categorization ---
cat_cols = [col for col in train_clean.columns if train_clean[col].dtype == 'object']
num_cols = [col for col in train_clean.columns if train_clean[col].dtype != 'object']
threshold = 3
woe_columns = list(s[s > 3].index)
one_hot_columns = list(s[s <= 3].index)

# --- Pipeline Definition ---
pipeline_rf = Pipeline([
    ('cleaner', DataCleaner(
        fill_strategies={'GarageYrBlt': 'value'},
        default_strategy='mode',
        drop_threshold=0.8,
        verbose=False
    )),
    ('preprocessor', CustomPreprocessor(woe_columns=woe_columns, one_hot_columns=one_hot_columns)),
    ('type_checker', TypeChecker()),  # Optional: safe type validation
    ('correlation_filter', SafeCorrelationFilter(threshold=0.8)),
    ('regressor', RandomForestRegressor(random_state=42, n_jobs=-1))
])

# --- Custom Scorer (log RMSE) ---
def log_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(np.log1p(y_true), np.log1p(y_pred)))

log_rmse_scorer = make_scorer(log_rmse, greater_is_better=False)

# --- Grid Search Parameters ---
param_grid = {
    'regressor__n_estimators': [10, 15, 20],
    'regressor__max_depth': [50, 100, None],
    'regressor__min_samples_split': [2, 5]
}

# --- GridSearchCV ---
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    pipeline_rf,
    param_grid=param_grid,
    cv=kfold,
    scoring=log_rmse_scorer,
    verbose=2,
    return_train_score=True
)

grid_search.fit(X_train, y_train)

best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)
rmse = log_rmse(y_test, y_pred)
y_pred_original_scale = np.expm1(y_pred)

print(f"\nBest Params: {grid_search.best_params_}")
print(f"Best CV Score (log RMSE): {grid_search.best_score_:.4f}")
print(f"Test RMSE: {rmse:.4f}")

with mlflow.start_run(run_name="LogisticRegression_Feature_Selection"):
    mlflow.log_params(grid_search.best_params_)
    mlflow.log_metric("cv_log_rmse", -grid_search.best_score_)
    mlflow.log_metric("test_log_rmse", rmse)
    mlflow.sklearn.log_model(best_rf, artifact_path="random_forest_model")
    print("\nMLflow run completed and metrics logged.")


Fitting 5 folds for each of 18 candidates, totalling 90 fits


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


***
Check Nans
[]
***
Check Nans
['Neighborhood_woe', 'Condition2_woe', 'Exterior1st_woe', 'GarageYrBlt_woe']
Neighborhood_woe Neighborhood woe NAmes 1.2869199656264364e-06
Condition2_woe Condition2 woe Norm -2.605896390959492e-09
Exterior1st_woe Exterior1st woe VinylSd -7.769675557074347e-07
GarageYrBlt_woe GarageYrBlt woe value 4.2124862333160545e-06
***
Check Nans
[]
[CV] END regressor__max_depth=50, regressor__min_samples_split=2, regressor__n_estimators=10; total time=   0.6s


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


***
Check Nans
[]
***
Check Nans
['BsmtCond_woe', 'HeatingQC_woe']
BsmtCond_woe BsmtCond woe TA -7.60299059935717e-09
HeatingQC_woe HeatingQC woe Ex -8.634770280845905e-07
***
Check Nans
[]
[CV] END regressor__max_depth=50, regressor__min_samples_split=2, regressor__n_estimators=10; total time=   0.6s


...

***
Check Nans
[]
***
Check Nans
['Condition2_woe', 'Exterior1st_woe', 'Exterior2nd_woe', 'ExterCond_woe', 'Functional_woe', 'GarageYrBlt_woe', 'GarageQual_woe', 'GarageCond_woe']
Condition2_woe Condition2 woe Norm -1.0795717658994379e-09
Exterior1st_woe Exterior1st woe VinylSd -8.66243357089302e-07
Exterior2nd_woe Exterior2nd woe VinylSd -8.884791374973195e-07
ExterCond_woe ExterCond woe TA -8.830308208157791e-08
Functional_woe Functional woe Typ -7.79042414281355e-08
GarageYrBlt_woe GarageYrBlt woe value 4.173076826057172e-06
GarageQual_woe GarageQual woe TA -3.215833318053588e-08
GarageCond_woe GarageCond woe TA -5.7046906826872773e-08
***
Check Nans
[]
[CV] END regressor__max_depth=None, regressor__min_samples_split=5, regressor__n_estimators=20; total time=   0.6s


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


***
Check Nans
[]
***
Check Nans
['RoofMatl_woe', 'Electrical_woe', 'GarageYrBlt_woe']
RoofMatl_woe RoofMatl woe CompShg 1.978151445288834e-08
Electrical_woe Electrical woe SBrkr -1.6186302498252008e-07
GarageYrBlt_woe GarageYrBlt woe value 4.2947155861470925e-06

Best Params: {'regressor__max_depth': 50, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 20}
Best CV Score (log RMSE): -0.1472
Test RMSE: 0.1534


2025/04/10 15:48:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



MLflow run completed and metrics logged.
🏃 View run LogisticRegression_Feature_Selection at: https://dagshub.com/goguaD/HW01.mlflow/#/experiments/0/runs/0cb1f977673d48bd9615a8d152bd9c04
🧪 View experiment at: https://dagshub.com/goguaD/HW01.mlflow/#/experiments/0


In [46]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

# --- Define log RMSE function ---
def log_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(np.log1p(y_true), np.log1p(y_pred)))

# --- Data Cleaning ---
cleaner = DataCleaner(
    fill_strategies={'GarageYrBlt': 'value'},
    default_strategy='mode',
    drop_threshold=1,
    verbose=False
)

x_cleaned = cleaner.fit_transform(X_train)
woe_columns = [col for col in woe_columns if col in x_cleaned.columns]
one_hot_columns = [col for col in one_hot_columns if col in x_cleaned.columns]

pipeline = Pipeline([
    ('cleaner', DataCleaner(
        fill_strategies={'GarageYrBlt': 'value'},
        default_strategy='mode',
        drop_threshold=0.8,
        verbose=False
    )),
    ('preprocessor', CustomPreprocessor(woe_columns=woe_columns, one_hot_columns=one_hot_columns)),
    ('regressor', LinearRegression())
])
pipeline.fit(X_train, np.log1p(y_train))
y_pred = np.expm1(pipeline.predict(X_test))
rmse = log_rmse(y_test, y_pred)
print(f"Test RMSE: {rmse:.4f}")
with mlflow.start_run(run_name="LinearRegression_LogRMSE"):
    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_metric("test_log_rmse", rmse)
    mlflow.sklearn.log_model(pipeline, artifact_path="linear_regression_model")
    print("MLflow run logged successfully.")


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


***
Check Nans
[]
***
Check Nans
['RoofMatl_woe', 'Electrical_woe', 'GarageYrBlt_woe']
RoofMatl_woe RoofMatl woe CompShg 2.3983549926358988e-05
Electrical_woe Electrical woe SBrkr -0.0002596540019032626
GarageYrBlt_woe GarageYrBlt woe value 0.00438722241077126
Test RMSE: 0.1377


2025/04/10 15:52:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


MLflow run logged successfully.
🏃 View run LinearRegression_LogRMSE at: https://dagshub.com/goguaD/HW01.mlflow/#/experiments/0/runs/aad24362c94c4f919520e111b5e50f2e
🧪 View experiment at: https://dagshub.com/goguaD/HW01.mlflow/#/experiments/0


In [47]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, mean_squared_error
import mlflow
import mlflow.sklearn

# --- Feature Categorization ---
cat_cols = [col for col in train_clean.columns if train_clean[col].dtype == 'object']
num_cols = [col for col in train_clean.columns if train_clean[col].dtype != 'object']
threshold = 3
woe_columns = list(s[s > 3].index)
one_hot_columns = list(s[s <= 3].index)
def log_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(np.log1p(y_true), np.log1p(y_pred)))

log_rmse_scorer = make_scorer(log_rmse, greater_is_better=False)
pipeline_gb = Pipeline([
    ('cleaner', DataCleaner(
        fill_strategies={'GarageYrBlt': 'value'},
        default_strategy='mode',
        drop_threshold=0.8,
        verbose=False
    )),
    ('preprocessor', CustomPreprocessor(woe_columns=woe_columns, one_hot_columns=one_hot_columns)),
    ('type_checker', TypeChecker()),
    ('correlation_filter', SafeCorrelationFilter(threshold=0.8)),
    ('regressor', GradientBoostingRegressor(random_state=42))
])
param_grid = {
    'regressor__n_estimators': [100, 200],
    'regressor__learning_rate': [0.05, 0.1],
    'regressor__max_depth': [3, 5],
    'regressor__min_samples_split': [2, 5]
}
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    pipeline_gb,
    param_grid=param_grid,
    cv=kfold,
    scoring=log_rmse_scorer,
    verbose=2,
    return_train_score=True
)
grid_search.fit(X_train, y_train)
best_gb = grid_search.best_estimator_
y_pred = best_gb.predict(X_test)
rmse = log_rmse(y_test, y_pred)
y_pred_original_scale = np.expm1(y_pred)

print(f"\nBest Params: {grid_search.best_params_}")
print(f"Best CV Score (log RMSE): {grid_search.best_score_:.4f}")
print(f"Test RMSE: {rmse:.4f}")
with mlflow.start_run(run_name="GradientBoosting_GridSearch"):
    mlflow.log_params(grid_search.best_params_)
    mlflow.log_metric("cv_log_rmse", -grid_search.best_score_)
    mlflow.log_metric("test_log_rmse", rmse)
    mlflow.sklearn.log_model(best_gb, artifact_path="gradient_boosting_model")
    print("MLflow run logged successfully.")


Fitting 5 folds for each of 16 candidates, totalling 80 fits


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


***
Check Nans
[]
***
Check Nans
['Neighborhood_woe', 'Condition2_woe', 'Exterior1st_woe', 'GarageYrBlt_woe']
Neighborhood_woe Neighborhood woe NAmes 1.2869199656264364e-06
Condition2_woe Condition2 woe Norm -2.605896390959492e-09
Exterior1st_woe Exterior1st woe VinylSd -7.769675557074347e-07
GarageYrBlt_woe GarageYrBlt woe value 4.2124862333160545e-06
***
Check Nans
[]
[CV] END regressor__learning_rate=0.05, regressor__max_depth=3, regressor__min_samples_split=2, regressor__n_estimators=100; total time=   1.0s


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


***
Check Nans
[]
***
Check Nans
['BsmtCond_woe', 'HeatingQC_woe']
BsmtCond_woe BsmtCond woe TA -7.60299059935717e-09
HeatingQC_woe HeatingQC woe Ex -8.634770280845905e-07
***
Check Nans
[]
[CV] END regressor__learning_rate=0.05, regressor__max_depth=3, regressor__min_samples_split=2, regressor__n_estimators=100; total time=   1.0s


...

***
Check Nans
[]
***
Check Nans
['Condition2_woe', 'Exterior1st_woe', 'Exterior2nd_woe', 'ExterCond_woe', 'Functional_woe', 'GarageYrBlt_woe', 'GarageQual_woe', 'GarageCond_woe']
Condition2_woe Condition2 woe Norm -1.0795717658994379e-09
Exterior1st_woe Exterior1st woe VinylSd -8.66243357089302e-07
Exterior2nd_woe Exterior2nd woe VinylSd -8.884791374973195e-07
ExterCond_woe ExterCond woe TA -8.830308208157791e-08
Functional_woe Functional woe Typ -7.79042414281355e-08
GarageYrBlt_woe GarageYrBlt woe value 4.173076826057172e-06
GarageQual_woe GarageQual woe TA -3.215833318053588e-08
GarageCond_woe GarageCond woe TA -5.7046906826872773e-08
***
Check Nans
[]
[CV] END regressor__learning_rate=0.1, regressor__max_depth=5, regressor__min_samples_split=5, regressor__n_estimators=200; total time=   2.2s


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


***
Check Nans
[]
***
Check Nans
['RoofMatl_woe', 'Electrical_woe', 'GarageYrBlt_woe']
RoofMatl_woe RoofMatl woe CompShg 1.978151445288834e-08
Electrical_woe Electrical woe SBrkr -1.6186302498252008e-07
GarageYrBlt_woe GarageYrBlt woe value 4.2947155861470925e-06

Best Params: {'regressor__learning_rate': 0.1, 'regressor__max_depth': 3, 'regressor__min_samples_split': 5, 'regressor__n_estimators': 200}
Best CV Score (log RMSE): -0.1268
Test RMSE: 0.1317


2025/04/10 15:56:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


MLflow run logged successfully.
🏃 View run GradientBoosting_GridSearch at: https://dagshub.com/goguaD/HW01.mlflow/#/experiments/0/runs/4213cfd405414179a4f3b0fcb75800cc
🧪 View experiment at: https://dagshub.com/goguaD/HW01.mlflow/#/experiments/0


# Create submit file

In [49]:
dito = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [55]:
preds = pipeline.predict(dito)

preds = np.exp(preds)

sample_submission_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
sample_submission_df["SalePrice"] = preds
sample_submission_df.to_csv("submission03.csv", index=False)
print("Final submission file 'submission03.csv' is ready!")

***
Check Nans
['GarageYrBlt_woe']
GarageYrBlt_woe GarageYrBlt woe value 0.00438722241077126
Final submission file 'submission03.csv' is ready!
